In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import joblib

# Load Dataset
df = pd.read_csv("step_data.csv")  # replace with your latest file name




In [3]:


# Select important features
selected_features = ["steps", "activity_duration", "step_frequency", "rest_period", "noise_flag"]
X = df[selected_features]
y = df["anomaly_detected"]

# Scale Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape for LSTM [samples, time_steps, features]
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build LSTM Model
model = Sequential([
    LSTM(64, activation='relu', return_sequences=True, input_shape=(1, X_train.shape[2])),
    Dropout(0.2),
    LSTM(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train Model
history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test), callbacks=[early_stop])

# Save model and scaler
model.save("lstm_anomaly_model.h5")
joblib.dump(scaler, "feature_scaler.pkl")


Epoch 1/50


c:\Users\sanke\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.8755 - loss: 0.6316 - val_accuracy: 0.9350 - val_loss: 0.3127
Epoch 2/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9065 - loss: 0.3238 - val_accuracy: 0.9350 - val_loss: 0.2531
Epoch 3/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9136 - loss: 0.2984 - val_accuracy: 0.9350 - val_loss: 0.2575
Epoch 4/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8888 - loss: 0.3489 - val_accuracy: 0.9350 - val_loss: 0.2578
Epoch 5/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9126 - loss: 0.2959 - val_accuracy: 0.9350 - val_loss: 0.2640
Epoch 6/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8992 - loss: 0.3306 - val_accuracy: 0.9350 - val_loss: 0.2622
Epoch 7/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9179 - loss: 0.2744 - val_accuracy: 0.9350 - val_loss: 0.2651


['feature_scaler.pkl']